In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
for i in range(50,60):
    print(train_data.text.iloc[i])
    print(train_data.target.iloc[i])

Deputies: Man shot before Brighton home set ablaze http://t.co/gWNRhMSO8k
1
Man wife get six years jail for setting ablaze niece
http://t.co/eV1ahOUCZA
1
SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf
0
Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze http://t.co/pcXarbH9An
1
Noches El-Bestia '@Alexis_Sanchez: happy to see my teammates and training hard ?? goodnight gunners.?????? http://t.co/uc4j4jHvGR'
0
#Kurds trampling on Turkmen flag later set it ablaze while others vandalized offices of Turkmen Front in #Diyala http://t.co/4IzFdYC3cg
1
TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE OR TAMBO INTL. CARGO SECTION. http://t.co/8kscqKfKkF
1
Set our hearts ablaze and every city was a gift And every skyline was like a kiss upon the lips @Û_ https://t.co/cYoMPZ1A0Z
0
They sky was ablaze tonight in Los Angeles. I'm expecting IG and FB to be filled with sunset shots if I know my p

In [6]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


Some very basic cleanup work to start.  Mostly just set all URLs and @terms to respective keywords

In [7]:
from string import punctuation
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import emoji

stopWords = set(stopwords.words('english'))

def clean_text(text):
    # change URL/@ tags to keywords
    text = ' '.join(['URL' if 'http://' in i or 'https://' in i else i for i in text.split(' ') ])
    text = ' '.join(['ATUSER' if len(i)>0 and i[0]=='@' else i for i in text.split(' ') ])
    #remove punctuation
    text = ''.join([c for c in text if c not in punctuation])
    #replace newline with space
    text = re.sub('\n',' ',text)
    #remove extra spaces
    text = re.sub(' +', ' ', text)
    #remove stop words
    text = ' '.join([i for i in text.lower().split(' ') if i not in stopWords])
    return text

def cleanTweet(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+','',txt)
    txt = re.sub(r'#','',txt)
    txt = re.sub(r'RT : ','',txt)
    txt = re.sub(r'\n','',txt)
    # to remove emojis
    txt = re.sub(emoji.get_emoji_regexp(), r"", txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+','',txt)
    txt = re.sub(r"https?://\S+|www\.\S+","",txt)
    txt = re.sub(r"<.*?>","",txt)
    return txt  

def remove_punct(text):
    table=str.maketrans('','',punctuation)
    return text.translate(table)

def make_Lower(text):
    return str.lower(text)

#text = [clean_text(i) for i in train_data.text]
train_data.text = train_data.text.apply(cleanTweet)
train_data.text = train_data.text.apply(remove_punct)
train_data.text = train_data.text.apply(make_Lower)
text = train_data.text.tolist()

#text = [clean_text(i) for i in train_data.text]
test_data.text = test_data.text.apply(cleanTweet)
test_data.text = test_data.text.apply(remove_punct)
test_data.text = test_data.text.apply(make_Lower)
test_text = test_data.text.tolist()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\duran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#inspect text
text

['our deeds are the reason of this earthquake may allah forgive us all',
 'forest fire near la ronge sask canada',
 'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
 '13000 people receive wildfires evacuation orders in california ',
 'just got sent this photo from ruby alaska as smoke from wildfires pours into a school ',
 'rockyfire update  california hwy 20 closed in both directions due to lake county fire  cafire wildfires',
 'flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas',
 'im on top of the hill and i can see a fire in the woods',
 'theres an emergency evacuation happening now in the building across the street',
 'im afraid that the tornado is coming to our area',
 'three people died from the heat wave so far',
 'haha south tampa is getting flooded hah wait a second i live in south tampa what am i gonna do what am i gonna do fvck flooding',
 'raining 

In [9]:
all_words = ' '.join(text).split(' ') + ' '.join(test_text).split(' ')

In [10]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts,key=counts.get, reverse=True)
vocab_to_int = {word:i for i,word in enumerate(vocab,1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
tweet_ints = []
for tweet in text:
    tweet_ints.append([vocab_to_int[word] for word in tweet.split(' ')])
    
tweet_ints_test = []
for tweet in test_text:
    tweet_ints_test.append([vocab_to_int[word] for word in tweet.split(' ') if word in vocab_to_int.keys()])


In [11]:
#how big is vocab?
len(counts.keys())

19956

In [12]:
vocab[0:20]

['',
 'the',
 'a',
 'to',
 'in',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'it',
 'with',
 'that',
 'by',
 'at',
 'this',
 'from']

In [13]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

#run vocab through spacy
doc = nlp(' '.join(vocab))

#use .vector to obtain vector of dim 300 for each word (if not in spacy vocab, use all zeros)
embedding_matrix = np.zeros((len(vocab)+1,300))
for index in range(len(vocab)):
    embedding_matrix[index+1] = doc[index].vector

spacy_embedding_dim = embedding_matrix.shape

In [14]:
# outlier review stats
tweet_lens = Counter([len(x) for x in tweet_ints])
print("Zero-length reviews: {}".format(tweet_lens[0]))
print("Maximum length: {}".format(max(tweet_lens)))

Zero-length reviews: 0
Maximum length: 54


In [15]:
# pad tweets to equal legnths
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

seq_length = 54

features = pad_features(tweet_ints, seq_length=seq_length)
sub_features = pad_features(tweet_ints_test, seq_length=seq_length)

In [16]:
split_frac = 0.8

## split data into training, validation data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, test_x = features[:split_idx], features[split_idx:]
train_y, test_y = train_data.target.tolist()[:split_idx], train_data.target.tolist()[split_idx:]

print(train_x[0])
print(train_y[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0  100
 5642   21    2  833    6   19  236  139 2032 3635   68   40]
1


In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))
sub_data = TensorDataset(torch.from_numpy(np.array(sub_features)), torch.from_numpy(np.zeros(len(sub_features))))

# dataloaders
batch_size = 12

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)
sub_loader = DataLoader(sub_data, shuffle=True, batch_size=1,drop_last=True)

In [18]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([12, 54])
Sample input: 
 tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     1,     1,
             5,    42,    72,  1821,  1397,     5,     2,   161,    33,    26,
             3,   246,     5,  1146],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,   262,  7500,  1196,   503,     1,  4661,  1980,
          3579,  7501,  4300,   788],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0

In [19]:
train_on_gpu=torch.cuda.is_available()

In [20]:
import torch.nn as nn

class ClassifierRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(ClassifierRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.embedding.weight=nn.Parameter(torch.tensor(embedding_matrix,dtype=torch.float32))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,bidirectional=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
      
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        #batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)

        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers*2, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [21]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 800 #spacy_embedding_dim[1]
hidden_dim = 512
n_layers = 2

net = ClassifierRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

ClassifierRNN(
  (embedding): Embedding(19957, 800)
  (lstm): LSTM(800, 512, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [22]:
# loss and optimization functions
lr=0.00005

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [23]:
# training params

epochs = 10

counter = 0
print_every = 100
clip=4 # gradient clipping


# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in test_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: 0.731076... Val Loss: 0.675498
Epoch: 1/10... Step: 200... Loss: 0.718021... Val Loss: 0.665505
Epoch: 1/10... Step: 300... Loss: 0.888784... Val Loss: 0.665196
Epoch: 1/10... Step: 400... Loss: 0.661205... Val Loss: 0.664840
Epoch: 1/10... Step: 500... Loss: 0.706299... Val Loss: 0.655859
Epoch: 2/10... Step: 600... Loss: 0.471574... Val Loss: 0.625947
Epoch: 2/10... Step: 700... Loss: 0.500327... Val Loss: 0.574914
Epoch: 2/10... Step: 800... Loss: 0.601276... Val Loss: 0.585929
Epoch: 2/10... Step: 900... Loss: 0.344684... Val Loss: 0.543098
Epoch: 2/10... Step: 1000... Loss: 0.519084... Val Loss: 0.532335
Epoch: 3/10... Step: 1100... Loss: 0.522595... Val Loss: 0.551805
Epoch: 3/10... Step: 1200... Loss: 0.319523... Val Loss: 0.544539
Epoch: 3/10... Step: 1300... Loss: 0.596785... Val Loss: 0.524060


KeyboardInterrupt: 

In [24]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    #h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.522
Test accuracy: 0.747


In [25]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0
all_preds = []
batch_size = 1

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in sub_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # convert output probabilities to predicted class (0 or 1)

    pred = torch.round(output.squeeze()).tolist()  # rounds to the nearest integer
    all_preds.append(pred)



In [26]:
len(all_preds)

3263